Maxwell solver for PEC bodies (EFIE)
=============================
**keys**: Maxwell single layer potential, EFIE, indirect ansatz, MoM, PEC scattering

In [ ]:
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG, GMRes

Load the mesh:

In [ ]:
#hull = Box((0,0,0), (100,10,5))
#deck1 = Box((30,0,5), (50,5,5))
#deck2 = Box((70,0,5), (85,10,15))
#shape = hull + deck1 + deck2

#shape = Box((0,0,0), (1,1,1))
#mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=3, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(1)

mesh = Mesh("resources/ship_1.stl")
mesh.Curve(1)
Draw(mesh)
fesL2 = SurfaceL2(mesh,order=0)
print(fesL2.ndof)

Generate the finite element spaces:

In [ ]:
order = 3
fesHDiv = HDivSurface(mesh, order=order, complex=True)
uHDiv,vHDiv = fesHDiv.TnT() # H(div_Gamma) 

fesHCurl = HCurl(mesh, order=order, complex=True)
uHCurl, vHCurl = fesHCurl.TnT() # H(curl_Gamma)

print ("ndof HDiv = ", fesHDiv.ndof)
print ("ndof HCurl = ", fesHCurl.ndof)

Define incoming plane wave $\boldsymbol E_{\mathrm{inc}}$:

In [ ]:
a = -0.758
b = -0.453
c = -0.524
E_inc = CF( (-c, 0., a) ) * exp( -1j * (a * x + b * y + c * z))
kappa = sqrt( a*a + b*b + c*c)
print(kappa)

Let's compute the relative error in input data, i.e., in the given Dirichlet trace $\boldsymbol m$

In [ ]:
n = specialcf.normal(3)
m_exa = -Cross(Cross(n, E_inc), n)
m = GridFunction(fesHCurl)
m.Set(-E_inc, definedon=mesh.Boundaries(".*"), dual=True)
error = sqrt(Integrate(Norm(m_exa - m) ** 2, mesh, BND) / Integrate(Norm(m_exa) ** 2, mesh, BND))
print("relative L2-error in m: ", error)

Have a look at the Dirichlet trace $\boldsymbol m$ that induces the scattered electric field, for instance the first real component: 

In [ ]:
Draw(m[0].real, mesh, draw_vol=False, order=3, min=-0.5, max=0.5);

Consider the indirect ansatz for the unknown scattered electric field, i.e.,  

$$\boldsymbol E(\boldsymbol x) = S_E\left(\boldsymbol j_{\mathrm{efie}}\right)(\boldsymbol x)$$ 

and solve EFIE for unknown density $\;\boldsymbol j_{\mathrm{efie}}$: 

In [ ]:
# indirect ansatz, i.e., solution of V j_efie = M m
j_efie = GridFunction(fesHDiv)
rhs_efie = LinearForm( - E_inc* vHDiv.Trace() * ds(bonus_intorder=20)).Assemble()
with TaskManager():
    pre = BilinearForm(uHDiv.Trace() * vHDiv.Trace() * ds).Assemble().mat.Inverse(freedofs=fesHDiv.FreeDofs()) 
    V = MaxwellSingleLayerPotentialOperator(fesHDiv, kappa, intorder=16, leafsize=320, eta=0., eps=1e-8)
    GMRes(A=V.mat, pre=pre, b=rhs_efie.vec, x=j_efie.vec, tol=1e-8, maxsteps=5000, printrates=False)

In [ ]:
# have a look at the solution
Draw (j_efie[0].real, mesh, draw_vol=False, order=3, min=-0.15, max=0.15);

Alternatively, we can consider the direct ansatz, i.e., 

$$ \boldsymbol E(\boldsymbol x) = S_E\left( \boldsymbol j\right)(\boldsymbol x) + S_M\left( \boldsymbol m \right)(\boldsymbol x) $$

and solve for the Neumann trace of the scattered field $\boldsymbol E$, i.e., 

$$\boldsymbol j = \dfrac{1}{\kappa} \boldsymbol n \times \left( \nabla \times \boldsymbol E\right)$$ 

In [ ]:
# direct ansatz, i.e., solution of   V j = ( 1/2 M  - K) m 
j = GridFunction(fesHDiv)
with TaskManager(): 
    K = MaxwellDoubleLayerPotentialOperator(fesHCurl, fesHDiv, kappa, intorder=16, leafsize=320, eta=0., eps=1e-8)
    M = BilinearForm(uHCurl.Trace() * vHDiv.Trace()* ds(bonus_intorder=3)).Assemble()
    rhs = (( 0.5*M.mat- K.mat)* m.vec).Evaluate()  
    GMRes(A=V.mat, pre=pre, b=rhs, x=j.vec, tol=1e-8, maxsteps=5000, printrates=False)

In [ ]:
# have a look at the solution
Draw (j[0].real, mesh, draw_vol=False, order=3, min=-0.5, max=0.5);

**Check the solutions**

It must hold 

$$ \boldsymbol j_{\mathrm{efie}} = \boldsymbol j + \boldsymbol j_{\mathrm{inc}}, \quad \boldsymbol j_{\mathrm{inc}} = \dfrac{1}{\kappa} \boldsymbol n \left( \nabla \times \boldsymbol E_{\mathrm{inc}} \right) $$

Thus, we can check the consistency of the numerical solutions and this is what we do now: 

In [ ]:
# compute the Neumann trace of the incoming singal: 
curlE_inc = CF( (a*b - 0*c, -c*c - a*a, 0*a + c*b) ) * (-1j) * exp( -1j * (a * x + b * y + c * z))
jexa_inc = GridFunction(fesHDiv)
jexa_inc.Set( 1/kappa*Cross(n, curlE_inc), definedon=mesh.Boundaries(".*"), dual=True) # Projektion in HDiv !
Draw (jexa_inc[0].real, mesh, draw_vol=False, order=3, min = -0.5, max=0.5);

In [ ]:
# Check consistency: j_test == j_efie: 
j_test = GridFunction(fesHDiv)
j_test.Set(j+jexa_inc, definedon=mesh.Boundaries(".*"), dual=True) 
Draw (j_test[0].real, mesh, draw_vol=False, order=3, min=-0.5, max=0.5); # trace of total field
Draw (j_efie[0].real, mesh, draw_vol=False, order=3, min=-0.5, max=0.5); # solution of indirect formulation